## Zadanie:

Proszę spróbować przeprowadzić taki proces regularyzacji modeli dla innego, nietrywialnego zbioru uczącego.

In [5]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)
print('GPU Name : ',tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.17.0-dev20240315
GPU Name :  []


In [55]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split

print ("TensorFlow version: " + tf.__version__)
print ("Keras version: " + keras.__version__)
print('GPU Name : ',tf.config.list_physical_devices('GPU'))

from keras import models
from keras import layers
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
LABELS= ['0', '1']

TensorFlow version: 2.16.1
Keras version: 3.0.5
GPU Name :  []


### Dataset
For this excercice I'm going to use Hotel Reservations dataset.
The goal is to predict whether the booking was cancelled or not.
https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset

### Load data

In [56]:
data = pd.read_csv('HotelReservations.csv')
data.drop(['Booking_ID'], axis=1, inplace=True)
print(data.shape)
data.head()

(36275, 18)


,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


### Reshape data

In [57]:
data['booking_status'] = data['booking_status'].map({'Canceled': 1, 'Not_Canceled': 0})
# data['market_segment_type'] = data['market_segment_type'].map({'Online' : 1, 'Offline' : 0})
# data['type_of_meal_plan'] = data['type_of_meal_plan'].map({'Not Selected' : 0, 'Meal Plan 1' : 1, 'Meal Plan 2' : 2, 'Meal Plan 3' : 3})
# data['room_type_reserved'] = data['room_type_reserved'].str[-1].astype(int)
# data.head()

In [58]:
# data = pd.get_dummies(data, columns=['booking_status'], prefix='booking_status')
data = pd.get_dummies(data, columns=['market_segment_type'], prefix='market_segment')
data = pd.get_dummies(data, columns=['type_of_meal_plan'], prefix='meal_plan')
data['room_type_reserved'] = data['room_type_reserved'].str[-1].astype(int)
data.head()


,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,...,booking_status,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Offline,market_segment_Online,meal_plan_Meal Plan 1,meal_plan_Meal Plan 2,meal_plan_Meal Plan 3,meal_plan_Not Selected
0,2,0,1,2,0,1,224,2017,10,2,...,0,False,False,False,True,False,True,False,False,False
1,2,0,2,3,0,1,5,2018,11,6,...,0,False,False,False,False,True,False,False,False,True
2,1,0,2,1,0,1,1,2018,2,28,...,1,False,False,False,False,True,True,False,False,False
3,2,0,0,2,0,1,211,2018,5,20,...,1,False,False,False,False,True,True,False,False,False
4,2,0,1,1,0,1,48,2018,4,11,...,1,False,False,False,False,True,False,False,False,True


### Normalize data

```python
# min-max normalization:
df=(df-df.min())/(df.max()-df.min())
# mean normalization:
df=(df-df.mean())/df.std()
```

In [59]:
data_og = data.copy()

data = (data - data.mean()) / data.std()
data.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,...,booking_status,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Offline,market_segment_Online,meal_plan_Meal Plan 1,meal_plan_Meal Plan 2,meal_plan_Meal Plan 3,meal_plan_Not Selected
0,0.298889,-0.261467,0.217398,-0.144801,-0.178817,-0.506404,1.614874,-2.137440,0.839230,-1.555641,...,-0.698052,-0.058802,-0.104384,-0.242642,1.563811,-1.333155,0.550642,-0.316607,-0.011741,-0.405843
1,0.298889,-0.261467,1.365974,0.563964,-0.178817,-0.506404,-0.933688,0.467837,1.164974,-1.097998,...,-0.698052,-0.058802,-0.104384,-0.242642,-0.639446,0.750079,-1.816011,-0.316607,-0.011741,2.463936
2,-1.628953,-0.261467,1.365974,-0.853566,-0.178817,-0.506404,-0.980237,0.467837,-1.766723,1.419035,...,1.432519,-0.058802,-0.104384,-0.242642,-0.639446,0.750079,0.550642,-0.316607,-0.011741,-0.405843
3,0.298889,-0.261467,-0.931177,-0.144801,-0.178817,-0.506404,1.463590,0.467837,-0.789491,0.503751,...,1.432519,-0.058802,-0.104384,-0.242642,-0.639446,0.750079,0.550642,-0.316607,-0.011741,-0.405843
4,0.298889,-0.261467,0.217398,-0.853566,-0.178817,-0.506404,-0.433285,0.467837,-1.115235,-0.525945,...,1.432519,-0.058802,-0.104384,-0.242642,-0.639446,0.750079,-1.816011,-0.316607,-0.011741,2.463936


### Train - Validate - Test split

In [60]:
train_data, test_data, train_labels, test_labels = train_test_split(data.drop(columns = ['booking_status']), data['booking_status'], test_size=0.2, random_state=42)
val_data, test_data, val_labels, test_labels = train_test_split(test_data, test_labels, test_size=0.5, random_state=42)
test_data

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,...,no_of_special_requests,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Offline,market_segment_Online,meal_plan_Meal Plan 1,meal_plan_Meal Plan 2,meal_plan_Meal Plan 3,meal_plan_Not Selected
34727,-1.628953,-0.261467,1.365974,-0.853566,-0.178817,-0.506404,-0.980237,0.467837,-1.766723,1.419035,...,-0.788129,-0.058802,-0.104384,-0.242642,1.563811,-1.333155,0.550642,-0.316607,-0.011741,-0.405843
21428,0.298889,-0.261467,0.217398,-0.144801,-0.178817,-0.506404,2.382934,0.467837,-0.463746,0.160519,...,-0.788129,-0.058802,-0.104384,-0.242642,1.563811,-1.333155,0.550642,-0.316607,-0.011741,-0.405843
33234,2.226730,-0.261467,1.365974,-0.853566,-0.178817,-0.506404,0.614069,-2.137440,0.187742,0.732572,...,0.483754,-0.058802,-0.104384,-0.242642,1.563811,-1.333155,-1.816011,3.158405,-0.011741,-0.405843
11713,0.298889,-0.261467,-0.931177,-0.144801,-0.178817,-0.506404,-0.654393,0.467837,1.490718,-1.555641,...,-0.788129,-0.058802,-0.104384,4.121185,-0.639446,-1.333155,0.550642,-0.316607,-0.011741,-0.405843
4977,0.298889,-0.261467,-0.931177,-0.144801,5.592161,-0.506404,-0.887139,0.467837,0.187742,0.846982,...,1.755637,-0.058802,-0.104384,-0.242642,-0.639446,0.750079,-1.816011,-0.316607,-0.011741,2.463936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24520,0.298889,-0.261467,0.217398,-0.853566,-0.178817,-0.506404,-0.712580,0.467837,-1.440979,1.190214,...,0.483754,-0.058802,-0.104384,-0.242642,-0.639446,0.750079,0.550642,-0.316607,-0.011741,-0.405843
12514,2.226730,-0.261467,1.365974,-0.853566,-0.178817,3.065405,-0.363462,0.467837,0.839230,-0.754766,...,1.755637,-0.058802,-0.104384,-0.242642,-0.639446,0.750079,0.550642,-0.316607,-0.011741,-0.405843
21370,0.298889,-0.261467,-0.931177,-0.853566,-0.178817,0.207958,1.568325,0.467837,0.187742,-0.525945,...,-0.788129,-0.058802,-0.104384,-0.242642,-0.639446,0.750079,0.550642,-0.316607,-0.011741,-0.405843
15809,0.298889,-0.261467,1.365974,-0.853566,-0.178817,-0.506404,0.369686,-2.137440,0.187742,-1.670051,...,-0.788129,-0.058802,-0.104384,-0.242642,1.563811,-1.333155,-1.816011,3.158405,-0.011741,-0.405843


### Stats

In [61]:
def calculate_class_weight(labels):
    count_of_labels = len(labels)
    count_of_positives = labels.sum()
    count_of_negatives = count_of_labels - count_of_positives
    
    positive_weight = count_of_labels /  (2 * count_of_positives)
    negative_weight = count_of_labels / (2 * count_of_negatives)
    
    return {0: negative_weight, 1: positive_weight}

In [62]:
class_weight = calculate_class_weight(data_og['booking_status'])
print(1 / class_weight[0] + 1 / class_weight[1])
print(f"Class weights: {class_weight}")

2.0
Class weights: {0: 0.7436449364493645, 1: 1.5260832982751367}


In [63]:
print(f"Positive cases in total: {data_og['booking_status'].sum()}, negative cases in total: {len(data_og['booking_status']) - data_og['booking_status'].sum()}, % of positive cases: {data_og['booking_status'].sum() / len(data_og['booking_status']) * 100:.2f}%")

Positive cases in total: 11885, negative cases in total: 24390, % of positive cases: 32.76%


## Visualization functions

In [64]:
# Define the confusion matrix for the results
def show_confusion_matrix(validations, predictions, num_classes):
    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(num_classes, num_classes))
    hm = sns.heatmap(matrix,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True,
                fmt='d')
    plt.yticks(rotation = 0)  # Don't rotate (vertically) the y-axis labels
    hm.set_ylim(0, len(matrix))
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [65]:
def PlotThreeAccuracyComparison(acc1, val_acc1, acc2, val_acc2, acc3, val_acc3, lab1 = '1', lab2 = '2', lab3 = '3'):
    plt.clf()   # clear figure
    plt.rcParams['figure.figsize'] = (25.0, 5.0) # set default size of plots
    epochs = range(1, len(acc1) + 1)
    plt.plot(epochs, acc1, 'bo', label='Training accuracy for ' + lab1)
    plt.plot(epochs, val_acc1, 'b', label='Validation accuracy for ' + lab1)
    plt.plot(epochs, acc2, 'ro', label='Training accuracy for ' + lab2)
    plt.plot(epochs, val_acc2, 'r', label='Validation accuracy for ' + lab2)
    plt.plot(epochs, acc3, 'go', label='Training accuracy for ' + lab3)
    plt.plot(epochs, val_acc3, 'g', label='Validation accuracy for ' + lab3)
    plt.title('Comparison of Training and Validation Accuracies')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def PlotThreeLossComparison(loss1, val_loss1, loss2, val_loss2, loss3, val_loss3, lab1 = '1', lab2 = '2', lab3 = '3'):
    plt.clf()   # clear figure
    plt.rcParams['figure.figsize'] = (25.0, 5.0) # set default size of plots
    epochs = range(1, len(loss1) + 1)
    plt.plot(epochs, loss1, 'bo', label='Training loss for ' + lab1)
    plt.plot(epochs, val_loss1, 'b', label='Validation loss for ' + lab1)
    plt.plot(epochs, loss2, 'ro', label='Training loss for ' + lab2)
    plt.plot(epochs, val_loss2, 'r', label='Validation loss for ' + lab2)
    plt.plot(epochs, loss3, 'go', label='Training loss for ' + lab3)
    plt.plot(epochs, val_loss3, 'g', label='Validation loss for ' + lab3)
    plt.title('Comparison of Training and Validation Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
def visualize_history(history1, history2, history3, visualizer_function, field: str, label1: str, label2: str, label3: str):
    value1 = history1.history[field]
    value1_val = history1.history['val_' + field]
    value2 = history2.history[field]
    value2_val = history2.history['val_' + field]
    value3 = history3.history[field]
    value3_val = history3.history['val_' + field]

    plt.figure(figsize=(25, 5))
    visualizer_function(value1, value1_val, value2, value2_val, value3, value3_val, label1, label2, label3)

def visualize_accuracy(history1, history2, history3, label1: str, label2: str, label3: str):
    visualize_history(history1, history2, history3, PlotThreeAccuracyComparison, 'acc', label1, label2, label3)

def visualize_loss(history1, history2, history3, label1: str, label2: str, label3: str):
    visualize_history(history1, history2, history3, PlotThreeLossComparison, 'loss', label1, label2, label3)


In [66]:
def print_val_accuracies(history1, history2, history3):
    val_accuracy1 = history1.history['val_acc']
    val_accuracy2 = history2.history['val_acc']
    val_accuracy3 = history3.history['val_acc']

    argmax1 = np.argmax(val_accuracy1)
    argmax2 = np.argmax(val_accuracy2)
    argmax3 = np.argmax(val_accuracy3)

    print ('Max accuracy for model 1 is: ', val_accuracy1[argmax1], ', achieved in the ', argmax1 , 'epoch.')
    print ('Max accuracy for model 2 is: ', val_accuracy2[argmax2], ', achieved in the ', argmax2 , 'epoch.')
    print ('Max accuracy for model 3 is: ', val_accuracy3[argmax3], ', achieved in the ', argmax3 , 'epoch.')

## Create NN

In [67]:
def createNNModel(input_shape: int, h1: int = 16, h2: int = 16, h3: int = 16):
    model = models.Sequential()
    model.add(layers.Dense(h1, activation='relu', input_shape=(input_shape,)))
    model.add(layers.Dense(h2, activation='relu'))
    model.add(layers.Dense(h3, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    return model

In [68]:
model1 = createNNModel(train_data.shape[1], 16, 16, 16)
model2 = createNNModel(train_data.shape[1], 32, 32, 32)
model3 = createNNModel(train_data.shape[1], 64, 64, 64)

model3.summary()

/home/lenovo/Uni/DeepNeuralNetworks-Course/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_44 (Dense)                │ (None, 64)             │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,985 (39.00 KB)

 Trainable params: 9,985 (39.00 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
# train the model
history1 = model1.fit(train_data, train_labels, epochs=100, batch_size=512, validation_data=(val_data, val_labels), class_weight=class_weight)
history2 = model2.fit(train_data, train_labels, epochs=100, batch_size=512, validation_data=(val_data, val_labels), class_weight=class_weight)
history3 = model3.fit(train_data, train_labels, epochs=100, batch_size=512, validation_data=(val_data, val_labels), class_weight=class_weight)



Epoch 1/100


57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - acc: 0.0000e+00 - loss: 0.8531 - val_acc: 0.0000e+00 - val_loss: 0.1272
Epoch 2/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: 0.1814 - val_acc: 0.0000e+00 - val_loss: -0.8512
Epoch 3/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: -0.8775 - val_acc: 0.0000e+00 - val_loss: -2.8232
Epoch 4/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: -2.9559 - val_acc: 0.0000e+00 - val_loss: -7.2108
Epoch 5/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: -7.9846 - val_acc: 0.0000e+00 - val_loss: -15.2395
Epoch 6/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: -15.6888 - val_acc: 0.0000e+00 - val_loss: -29.1039
Epoch 7/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: -30.4079 - val_acc: 0.0000e+00 - val_loss: -49.7006
Epoch 8/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: -51.8245 - val_acc: 0.0000e+00 - val_loss: -

In [ ]:
visualize_accuracy(history1, history2, history3, 'Model 1', 'Model 2', 'Model 3')